# Set kernel to: `Python 3`
If this kernel is not available, see the Kernel Installation Instructions section located in at the bottom of this Notebook. 

# Data Science on the C3 AI Suite
## C3DS - Capstone
#### For questions, comments, or concerns, please email the C3.ai Academy Team \< <training@c3.ai> \>, discuss with your Trainer, or leave your thoughts in the Google Forms at the end of this module.

The goal of this capstone project is to give you more hands on experience with the C3 platform and type system, and to test your ability to leverage the platform to tackle data science problems.

The theme of the project is `Anomaly Detection` for `SmartBulbMeasurement`s. Over the course of the assignment you will do the following:

1. **Load** new data using `SmartBulbMeasurement_Shuffled.json`.
2. **Merge** anomalous `SmartBulbMeasurement` instances onto the environment.
3. **Create**, **evaluate**, **visualize**, and **compare** metrics to be used for anomaly detection.
4. **Create** an anomaly detection model in `Python` using a `OneClassSVM`.
5. **Write** an `MLSerialPipeline` that implements the same model in the platform.
6. **Write** a `MapReduce` job to train an analogous model for each `SmartBulb`.
7. **Write** a method for the `SmartBulbMeasurementSeries` type that automates the process of flagging anomalies.

A starter repo for the project has been built-on, by you, as you worked through the C3DS exercises. A key difference, however, is loading extra/distinct data (to wit, we will cover first in this capstone). Other versions of the repo can also be used, but you will need to add `c3training/lightbulbAD/src/MLTrainingJobSpecTemp.c3typ` and `c3training/lightbulbAD/seed/ActionRuntime/py-example.json` **if** they are missing.

## Capstone Prelude: Setup \& Installation
#### Package Installation:

In [ ]:
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import RobustScaler
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline

Before starting the assignment, you should check that the necessary `SmartBulb` and `SmartBulbMeasurement` data has been uploaded to the server. To do so, run `c3Grid(SmartBulb.fetch())` **and** `c3Grid(SmartBulbMeasurement.fetch())`, respectively, making sure the grid is populated appropreiately. 

If you are using a local server, you may prefer to work only with a subset of the data. In this case, it is sufficient to load the measurements from `SmartBulbMeasurementSmall/SmartBulbMeasurent8.csv` (SMBLB70 - SMBLB79).

In [ ]:
assert(c3.SmartBulb.fetchCount() == 100)

In [ ]:
assert(c3.SmartBulbMeasurement.fetchCount() > 0)

Start by loading the measurements from the file `SmartBulbMeasurement_Shuffled.json` and exploring the data.

## 1. Loading Capstone Data
First, upload `SmartBulbMeasurement_Shuffled.json` to your Jupyter tree. You can do this by navigating to your Jupyter **Home Page**, selecting the `Upload` button, and directly uploading the `*.json` file.

Once this have successfully uploaded, return to you console, and run the following command to:

> `c3Grid(JupyterFile.fetch({filter: "name == 'SmartBulbMeasurement_Shuffled.json'"}))`

If you see `no content`, the file was incorrectly uploaded. Delete and re-upload the file in your Jupyter tree. If you see a `c3Grid` populated appropriately, the file has successfully uploaded.

Now, run the cell below to extract the data. 

In [ ]:
fetchSpec = {"filter": "name == 'SmartBulbMeasurement_Shuffled.json'"}

## Use Python API to get file information
fetchJupyterFile = c3.JupyterFile.fetch(spec=fetchSpec)
c3.File(url=fetchJupyterFile.objs.toJson()[0]['externalFile']['url'])

In [ ]:
## Define the FetchSpec
fetchSpec = {"filter": "name == 'SmartBulbMeasurement_Shuffled.json'"}

## Use Python API to get file information
fetchJupyterFile = c3.JupyterFile.fetch(spec=fetchSpec)

## Process & Filter that file information
s3File = c3.File(url=fetchJupyterFile.objs.toJson()[0]['externalFile']['url'])

if s3File.exists():
    data = pd.read_json(s3File.readString())
    print('File Exists - Data Extracted.')
else:
    print('File DNE - Check syntax')

In [ ]:
print(data['id'])

## 2. Add Anomalous Measurements

We have created 100 anomalous `SmartBulbMeasurement`s for you by taking existing measurements and permuting values between fields (for example, swapping temperature and voltage). We'll use these measurements as a validation set for the anomaly detection models that we train later in the project.

#### How many (fake) measurements are there for SmartBulb SMBLB70?

In [ ]:
# Answer here

#### How many (real) measurments for SMBLB70 already exist on the platform?

You should explore the type files in the `src` folder in order to figure out how to construct the fetch.

In [ ]:
# Answer here

You may try something like the above, but get an error: "Filter by parent is required."

This is annoying, but it's a reasonable restriction for types that mix `TimeseriesDataPoint` as it prevents inefficient fetches. 

We can work around it as follows:

In [ ]:
# Fetch to find the appropriate SmartBulbMeasurementSeries id
# Alternatively, note that SmartBulbMeasurement ids are all of the form <parent_id>#...
c3.SmartBulbMeasurementSeries.fetch(spec={'filter': 'smartBulb.id == "SMBLB70"'}).objs[0].id

In [ ]:
c3.SmartBulbMeasurement.fetchCount(spec={'filter': 'parent == "SBMS_serialNo_SMBLB70"'})

Next we'll merge the fake measurements onto the environment. It's no accident that we stored our data in json (dictionary) format; if `data` is a list of dictionaries with key:value pairs defined as shown by
`c3ShowType(MyType)`, we can **upsert/merge** onto the environment using syntax like `MyType.mergeBatch(objs=data)`.

#### Merge the Data

In [ ]:
# Answer here

#### What's the difference between using merge and upsert?

Upsert completely overwrites existing instances. Merge updates existing instances, only overwriting values for the specified fields.

Both methods will create new instances for objects with new ids.

#### After merging, how many total measurements for SMBLB70 exist on the platform?

In [ ]:
# Answer here

**NOTE**: The core sections of the project (up until the MapReduce job) only need to be completed for a single smartbulb (`SMBLB70`). You're encouraged to experiment with other smartbulbs afterward, but it's more practical to start with a single example.

## 2. Create Metrics

Next we'll create and use metrics to visualize the data. Our goal is to identify metrics that can be used to find anomalies.

Data is available for `SMBLB70` on an hourly basis for the date range `"2011-01-01T04:00:00.000"` to `"2014-11-06T04:00:00.000"`.

First write code to compute `AveragePower` and visualize the result. Do this once for the entire date range and then again over a shorter window (~1 week).

In [ ]:
def eval_metric_results_to_df(emr): 
    ds = c3.Dataset.fromEvalMetricsResult(result = emr)
    df = c3.Dataset.toPandas(dataset = ds)
    df['source'] = df.index.str.split('_').str[0]
    df['timestamp'] = pd.to_datetime(df.index.str.split('_').str[1],format="%Y-%m-%dT%H:%M:%S.%f")
    return df

#### Computation + Visualization of  `AveragePower`

**Note:** The simple metric definition is only included to demonstrate syntax. `AveragePower` is already defined in `seed/SimpleMetric/AveragePower_SmartBulb.json`, so we don't need to redefine it here. 

In [ ]:
# Answer here

#### What is the significance of the spikes in the first plot? At what timestamps do they occur?

The spikes represent the anomalies that we added in the previous section. The power values for these measurements are too large to be true readings.

In [ ]:
# Answer here

#### Repeat the above (Computation + Visualization) for `AverageTemperature`, `AverageVoltage`, and `AverageLumens`

Do not worry about the results now, as we will reflect on them here shortly.

In [ ]:
def viz(metric, time_range):
    spec = c3.EvalMetricsSpec(
        ids = ["SMBLB70"],
        expressions = [metric],
        start = "2011-01-01T04:00:00.000",
        end = "2014-11-06T04:00:00.000" if time_range == 'long' else "2011-01-08T04:00:00.000",
        interval = "HOUR"
    )
    result = c3.SmartBulb.evalMetrics(spec=spec).toJson()
    df = eval_metric_results_to_df(result)
    df[df['source']=='SMBLB70'].plot(x='timestamp',y=metric)

In [ ]:
# Answer here: AverageTemperature, long

In [ ]:
# Answer here: AverageTemperature, short

In [ ]:
# Answer here: AverageVoltage, long

In [ ]:
# Answer here: AverageVoltage, short

In [ ]:
# Answer here: AverageLumens, long

In [ ]:
# Answer here: AverageLumens, short

#### Are there spikes in the plots for the full date range that don't represent anomalies? What's causing them?

The temperature plot drops down to 0 several times. These spikes represent timestamps with **missing values**.

In [ ]:
# Answer here

#### Which of the simple metrics explored above are good features for anomaly detection?

In descending order of usefulness: `AveragePower`, `AverageVoltage`, `AverageTemperature`.

The values for `AverageLumens` vary the most, so it's unlikely that anomalous values will stand out. From the plots above, it may seem that we should reject `AverageTemperature` as well, but let's err on the side of not throwing out information; this feature would be useful if `Temperature` and `Lumens` were swapped, even though this doesn't seem to have happened for `SMBLB70`.

**Note:** Even though we're making a practical decision not to use `AverageLumens`, you should be skeptical of the idea that it's a useless feature. `AverageVoltage` and `AverageLumens` are highly correlated (why?), so we should be suspicious if one is high and the other is low, even if neither value is unusual individually.

Finally, let's try to improve on the above by defining some compound metrics based on `AverageTemperature`.

* First, write a metric that computes a smoothed average of `AverageTemperature` (remember the `window` function? If not, use the console to investigate.)
* Then write a metric to compute the difference between `AverageTemperature` and `SmoothedAverageTemperature`

Roughly speaking, the goal is to decompose `AverageTemperature` values as "typical value" + "deviation" so that large deviations stand out more.

#### Write, evaluate, and visualize the metrics described above

In [ ]:
# Answer here

Now that we've reduced the noise, we can see one of the anomalies that wasn't visible before.

In [ ]:
# Answer here

Try tuning the first metric based on your visualizations of the second. Some parameters are much better than others.

You're encouraged to explore and create additional metrics (and to improve upon the model defined in the next section), but you should try to complete the rest of the assignment first.

## 3. Anomaly Detection Model in Python

Next, we will train an anomaly detection model in Python using a One-Class Support Vector Machine. This algorithm is a generalization of [Support Vector Machines used for classification](https://towardsdatascience.com/introduction-to-support-vector-machine-svm-4671e2cf3755) that learns a decision boundary separating typical points and outliers for a given data set. After training the model, we will flag points that are far outside the decision boundary as potential anomalies.

First, generate a data set to train the model with. For reproducibility, please use only the metrics `AveragePower`, `AverageTemperature` and `AverageVoltage` for `SMBLB70` to generate features. Use the full date range from the previous section; this is an unsupervised approach, so we **do not** need a separate test set.

#### Generate Training Set

In [ ]:
# Answer here

Next, build a Scikit-learn pipeline and fit it to the data. Your pipeline should have two components:

* [RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html) - Our anomaly detection algorithm will perform better with scaled data. `RobustScaler` is preferable to `StandardScaler` because it is less sensitive to outliers and we have already polluted our data with fake measurements.
* [OneClassSVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html) - The algorithm described above. For reproducibility, please use the hyperparameter value $\nu = 0.01$ (which means that roughly 99% of normal points should fall within the decision boundary). Note that the `.decision_function` method of the trained model reflects the distance between each point and the decision boundary; this is what we will ulitmately use to label anomalies.

#### Build + Train Model

In [ ]:
# Answer here

Now let's see if we can identify anomalies using the trained model. Evaluate the model's decision function on the training set and visualize the results.

#### Make a histogram of `decision_function` values

In [ ]:
# Answer here

#### How many timestamps have a `decision_function` value less than -10?  How many are anomalies?

8 timestamps have a `decision_function` value less than -10. This includes all 4 anomalies that we added in the first section. The other 4 timestamps have missing data. In sections 5 and 6, we'll make a point of removing rows with missing data before training or applying models. 

In [ ]:
# Answer here

In practice, we will predict that a measurement is anomalous whenever its `decision_function` value is below some threshold $\tau$. We have a lot of freedom in choosing $\tau$, but ideally want to tune it so that our fake measurements stand out (this is an example of [semi-supervised learning](https://en.wikipedia.org/wiki/Semi-supervised_learning)). We've chosen $\tau = -10$ for `SMBLB70`, but keep in mind that we may need different parameters for different smartbulbs.

## 4. Reproduce Model in Platform Code

Next, rewrite the model from the previous step as an `MLSerialPipeline` and repeat training/prediction using the new version.

> Note: You must include an extra argument (`modelCategoryOverride = "classifier"`) when defining the `SklearnPipe` for `OneClassSVM` since it is an unsupervised algorithm. This is an idiosyncrasy of platform version 7.9 and is necessary to avoid an error when attempting to upsert the model.

#### Convert Model to `MLSerialPipeline`

In [ ]:
# Answer here

Keep in mind that training and prediction will require `c3.Dataset` objects (`c3ShowType(Dataset)`), so you may need to do some conversion depending on how you generated data in the previous section.

#### Train Model
As you could have guessed, this may take some time so please be patient.

In [ ]:
# Answer here

We'll also take this opportunity to practice upserting to, and fetching from, the platform, even though it isn't strictly necessary.

#### Upsert the trained model and then retrieve it using a get or fetch command
> Note: When fetching the model, make sure to grab the right model ID from the returned obj

In [ ]:
# Answer here

#### Run the model's process method on the data. Do you get the same values as in the previous section?

In [ ]:
# Answer here

## 5. MapReduce

Write and run a **MapReduce** job that trains **and** upserts a separate anomaly detection model for each smartbulb.

You shouldn't try to write the job from scratch; try adapting [this](https://c3iot.app.box.com/file/523449242920)  example.

If you are using a local server (or just need to go faster), try limiting the job to run only on `SMBLB70`, `SMBLB71` , and `SMBLB72`.

Some useful commands for debugging: `c3Grid(job.status())`, `c3QErrs(MapReduceQueue)`, and `job.cancel()`. Also keep in mind that code in your map and reduce functions will give more transparent error messages if run as snippets outside of a job.

Your **main objective** is simply to write a job that completes successfully, but you should also consider correctness from a data science perspective as much as time allows:
* `Dataset.fromEvalMetricsResult` orders features alphabetically, unless a second argument is passed whereas `Dataset.extractColumns` (used in the example MR job) will match the order of its input. Be sure that column order is consistent accross **all** of your code.
* Start and end dates like `DateTime(2011, 1, 1)` and `"2011-01-01"` will give slightly different results.
* If we use the same time window for all smartbulbs, our `EvalMetricsResult`s will include lots of timestamps with no data. We should filter these out before training each model.

#### Write and run the MapReduce Job

In [ ]:
# Write & execute the job in your browser's JS Console / sources tab (i.e., use snippets)
#NOTE: For assessment, log snippet, here.

#### Fetch and test the model trained for SMBLB70. Does it agree with the model you trained locally?

In [ ]:
# Answer here

This shouldn't match our local model since we've gotten more careful about filtering out missing data. However, if we do the same for our local pipeline, then the models will agree:

In [ ]:
# Answer here

## 6. Write a Method to Flag Anomalies

Now that we have a trained model for each smartbulb, we can flag potential anomalies for each `SmartBulbMeasurementSeries`. 

The workflow might look something like this:

1. **Evaluate** metrics on the `SmartBulbMeasurementSeries` to generate a data set `X`.
2. **Fetch** the trained model `m` for the corresponding smartbulb.
3. **Compute** `decision_function(X)`.
4. **Flag** all measurements for which the value of `decision_function` is below some **pre-defined** threshold.
5. Make a **record** containing our list of potential anomalies.

It would be inefficient to have a Data Scientist spin up a notebook and work through these steps every time we want to make predictions, so let's wrap up our project by writing a method that automates this workflow.

Start by adding a field `anomalousMeasurements: [SmartBulbMeasurement]` to the `SmartBulbMeasurementSeries` type that can be used to store a record of flagged/potential anomalies.

> Note: This isn't quite copy paste; do you remember the complete syntax for fields that store data? Read any error messages carefully if provisioning fails.

#### Add `anomalousMeasurements` to the `SmartBulbMeasurementSeries` type

In [ ]:
# Edit SmartBulbMeasurementSeries.c3typ and re-provision
#NOTE: For assessment, log / comment implemented changes, here.

Now write a method attached to the same type that performs all of the steps described above and updates the `anomalousMeasurements` field to be the resulting list of flagged measurements.

Some things to keep in mind:

* The final step is non-trivial since we need to recover `SmartBulbMeasurment`s from timestamps. You may want to prototype this in the notebook starting with `SMBLB70`.
* The `decision_function` threshold $\tau = -10$ used before may not be appropriate for all smartbulbs, so the reference solution uses the heuristic <br>`tao = -3 * decision_function(X).std()` instead. You're welcome to improve on this if time allows, but remember that rapid prototyping is almost always preferable to premature optimization.
* As with the MapReduce job, be sure that you are using the correct data. Make sure that data formatting (e.g. column/feature order) is consistent with all previous work and that your code will not be broken by missing data.

In terms of syntax, remember that you can specify a Python runtime by using a decorator like `@py(env='example')` in the method declaration. If you want to be fancy, try implementing this method as a member function. (See [these](https://community.c3.ai/t/what-is-a-member-function/547) [posts](https://community.c3.ai/t/how-to-access-this-in-python-member-function/2363) for more details.)

#### Add `updateAnomalousMeasurements` method to the `SmartBulbMeasurementSeries` type

In [ ]:
# Edit SmartbulbMeasurementSeries.c3typ, & add SmartBulbMeasurementSeries.py; now re-provision
#NOTE: For assessment, log / comment implemented changes, here.

#### Fetch a `SmartBulbMeasurementSeries` for SMBLB70 and run `updateAnomalousMeasurements`

In [ ]:
# Answer here

The point of implementing our method as a member function is that we can use the syntax `series.updateAnomalousMeasurements()`, but this may not be available if the instance `series` is older than the method. In this case, we can fall back on the following:

In [ ]:
c3.SmartBulbMeasurementSeries.updateAnomalousMeasurements(this = series)

#### Now fetch again and check the `anomalousMeasurements` field. Is this the result you expected?

In [ ]:
# Answer here

It looks like we've flagged some measurements that aren't anomalies, but this is only because there are duplicate timestamps in the data. Although it may feel like a bug, it's better than not knowing that a data issue exists, and it doesn't mean that our model has flagged any timestamps incorrectly.  

#### Repeat the above for SMBLB72. Did we catch the fake measurement(s) merged in step 1?

In [ ]:
# Answer here

Our model gets it right. This time there are no duplicate measurements, so it's less ambiguous.

## Conclusion

Congratulations! You have successfully completed the C3DS Capstone Assignment! 

In this assignment, you've merged anomalous `SmartBulbMeasurement` instances onto the environment, and used metrics to create, evaluate, visualize, and compare results to be used for anomaly detection. Then, you created an anomaly detection model in Python using a `OneClassSVM`, wrote a an `MLSerialPipeline` that implements this model on the platform, and wrote a `MapReduce` job to train a similar model for each SmartBulb! Finally, you automated the detection/flagging pipeline by writing a method for `SmartBulbMeasurementSeries`. 

To recieve credit 

## Make sure to CLOSE AND HALT  (from bottom of the File menu at the top) this notebook when you leave.

## Appendix - Kernel Installation Instructions <a class="anchor" id="apdx">

If the **`Python 3`** kernel is not available, perform the following:
1. Select the 'Kernel' tab at the top of this notebook, then select 'Manage Kernels';
2. Scroll to find `Python 3`, and click the 'Install' button;
> Note: This installation will "fail", but don't worry, exit the Manage Kernels UI and return to the main notebook.
3. Navigate to the 'File' tab located at the top of the notebook, and select 'Close and Hault'. This will close the notebook. Re-open it.
4. You should notice, in the upper-right corner, the kernel is now set correctly to `Python 3`. If it is not, navigate to the Kernel tab, then select 'Change Kernel' and select the appropriate option.